In [22]:
# import pandas as pd
# import pulp as pl

# # Veriyi okuma
# df1 = pd.read_csv("set1_new.csv")
# df2 = pd.read_csv("set2.csv")

# color_texture_combinations = df1[['Renk', 'Kıvam']].drop_duplicates().values.tolist()

# print(color_texture_combinations)

In [28]:
import pandas as pd
import pulp as pl
from collections import defaultdict
import random



def find_mandatory_dish_id(df, used_dishes):
    for dish_name in mandatory_dish_names:
        dish_row = df[df['Yiyecek adı'] == dish_name]
        if not dish_row.empty:
            dish_id = dish_row.iloc[0]['ID']
            if dish_id not in used_dishes:
                return dish_id, dish_name
    return None, None


df1 = pd.read_csv("set1_new.csv")
df2 = pd.read_csv("set2.csv")

age = 20
gender = "Kadın"

limits = df2[(df2["Cinsiyet"] == gender) & (df2["Yaş Grubu"].apply(lambda x: age in range(*map(int, x.split('-')))))].iloc[0]

all_mandatory_dish_names = ["Etli Nohut", "Etli Kuru Fasulye", "Zeytinyağlı Barbunya", "Börülce Salatası", "Mercimek Salatası", "Piyaz"]

mandatory_dish_names = random.sample(all_mandatory_dish_names, 4)
print(f"Zorunlu yemekler: {mandatory_dish_names}")
mandatory_dish_ids = [df1[df1['Yiyecek adı'] == name].iloc[0]['ID'] for name in mandatory_dish_names if not df1[df1['Yiyecek adı'] == name].empty]

categories = {
    "main": range(1, 72),
    "soup": range(72, 100),
    "half_main": range(100, 150),
    "dessert_salad": range(150, 210),
}

price_conversion = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}

used_dishes = set()
used_mandatory_dishes = set()

df1['Renk'] = df1['Renk'].str.lower()
df1['Kıvam'] = df1['Kıvam'].str.lower()


available_colors = df1['Renk'].drop_duplicates().values.tolist()
available_textures = df1['Kıvam'].drop_duplicates().values.tolist()

def check_menu(menu, df1):
    # Menüdeki yemeklerin kıvam ve renklerini saklayacak iki liste oluşturalım.
    textures_in_menu = [df1.set_index("ID").loc[dish_id]["Kıvam"] for dish_id in menu]
    colors_in_menu = [df1.set_index("ID").loc[dish_id]["Renk"] for dish_id in menu]
    
    # Her bir kıvam için yemek sayısını kontrol edelim.
    for texture in set(textures_in_menu):
        count = textures_in_menu.count(texture)
        if count > 2:
            return f"Menüde {texture} kıvamında {count} tane yemek bulunmaktadır!"
    
    # Her bir renk için yemek sayısını kontrol edelim.
    for color in set(colors_in_menu):
        count = colors_in_menu.count(color)
        if count > 2:
            return f"Menüde {color} renkli {count} tane yemek bulunmaktadır!"
    
    return "Menü kısıtlamalara uygundur."



for week in range(1, 5):
    print(f"{week}. Hafta Menüleri\n")
    mandatory_dish_day = random.choice(range(1, 6))
    mandatory_dish_id, mandatory_dish_name = None, None
    
    for day in range(1, 6):
        prob = pl.LpProblem(f"Menu_Optimization_{week}_{day}", pl.LpMinimize)
        available_dishes = list(set(df1['ID']) - used_dishes)
        if day != mandatory_dish_day:
            available_dishes = [dish for dish in available_dishes if dish not in mandatory_dish_ids]
        dish_vars = pl.LpVariable.dicts("Dish", available_dishes, 0, 1, pl.LpBinary)

        prob += pl.lpSum([price_conversion[df1.set_index("ID").loc[i]["Fiyat"]] * dish_vars[i] for i in available_dishes])
        
        for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
            lower_limit = float(limits[nutrient].split('-')[0])
            upper_limit = float(limits[nutrient].split('-')[1])
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintGE,
                name=f"{nutrient}_lower_bound",
                rhs=lower_limit
            )
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintLE,
                name=f"{nutrient}_upper_bound",
                rhs=upper_limit
            )

        for cat, rng in categories.items():
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in rng]) == 1
 
        for color in available_colors:
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if df1.set_index("ID").loc[i]["Renk"] == color]) <= 2
        for texture in available_textures:
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if df1.set_index("ID").loc[i]["Kıvam"] == texture]) <= 2

        if day == mandatory_dish_day:
            mandatory_dish_id, mandatory_dish_name = find_mandatory_dish_id(df1, used_mandatory_dishes)
            if mandatory_dish_id is None:
                print("Uygun zorunlu yemek kalmadı.")
            else:
                used_mandatory_dishes.add(mandatory_dish_id)
                prob += dish_vars[mandatory_dish_id] == 1
                used_dishes.add(mandatory_dish_id)  # Zorunlu yemek artık kullanılmıştır.
                print(f"Zorunlu yemek {mandatory_dish_id} ({mandatory_dish_name}) hafta {week}, gün {day} için eklendi.")

        
        prob.solve(pl.PULP_CBC_CMD(msg=False))
        print("Status:", pl.LpStatus[prob.status])

        selected_dishes_for_today = [int(v.name.split('_')[1]) for v in prob.variables() if v.varValue == 1]

        # Menüyü kontrol edelim.
        menu_validation_result = check_menu(selected_dishes_for_today, df1)
        print(menu_validation_result)

        print(f"{day}. Gün Menüsü:")

        total_nutrients = defaultdict(float)
        
        for v in prob.variables():
            if v.varValue == 1:
                dish_id = int(v.name.split('_')[1])
                dish_info = df1.set_index("ID").loc[dish_id]
                print(f"ID: {dish_id}, Yemek: {dish_info['Yiyecek adı']}, Fiyat: {dish_info['Fiyat']}, Renk: {dish_info['Renk']}, Kıvam: {dish_info['Kıvam']}")
                used_dishes.add(dish_id)
                
                for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
                    total_nutrients[nutrient] += dish_info[nutrient]
        
        print("Toplam Besin Değerleri:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")
        
        print("---")


Zorunlu yemekler: ['Etli Nohut', 'Börülce Salatası', 'Etli Kuru Fasulye', 'Mercimek Salatası']
1. Hafta Menüleri

Status: Optimal
Menü kısıtlamalara uygundur.
1. Gün Menüsü:
ID: 134, Yemek: Fırında Makarna, Fiyat: e, Renk: sarı, Kıvam: katı
ID: 186, Yemek: Mürdüm Eriği, Fiyat: e, Renk: kırmızı, Kıvam: katı
ID: 71, Yemek: Kremalı Domates Çorbası, Fiyat: e, Renk: kırmızı, Kıvam: sıvı
ID: 80, Yemek: İşkembe Çorbası, Fiyat: e, Renk: beyaz, Kıvam: sıvı
Toplam Besin Değerleri:
Enerji: 584.2
Karbonhidrat: 60.3
Protein: 23.2
Yağ: 26.6
Lif: 8.4
---
Status: Optimal
Menü kısıtlamalara uygundur.
2. Gün Menüsü:
ID: 128, Yemek: Zeytinyağlı Kereviz, Fiyat: e, Renk: beyaz, Kıvam: yumuşak
ID: 199, Yemek: İrmik Helvası, Fiyat: e, Renk: kahverengi, Kıvam: yumuşak
ID: 36, Yemek: Sebzeli Tavuk Sote, Fiyat: d, Renk: kırmızı, Kıvam: katı
ID: 97, Yemek: Yulaf Özü Çorbası, Fiyat: e, Renk: beyaz, Kıvam: sıvı
Toplam Besin Değerleri:
Enerji: 597.5
Karbonhidrat: 62.2
Protein: 25.299999999999997
Yağ: 27.1
Lif: 9.0
